In [1]:
import requests

In [3]:
# fetch a web page
r = requests.get("https://www.lequipe.fr/")
print(r.text[:1000])

<!DOCTYPE html><html class="no-js" lang="fr"><head><base href="/"><title>L'Équipe - L'actualité du sport en continu.</title><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no"><meta name="google-site-verification" content="42ztUq_6M1oIgM8mQ5XOzhmpA_LqDFmInnBEc2gGYY4"><link rel="icon" href="/img/favicons/favicon.svg"><link rel="icon" type="image/png" href="/img/favicons/favicon-16x16.png" sizes="16x16"><link rel="icon" type="image/png" href="/img/favicons/favicon-32x32.png" sizes="32x32"><link rel="icon" type="image/png" href="/img/favicons/favicon-96x96.png" sizes="96x96"><link rel="apple-touch-icon" sizes="57x57" href="/img/favicons/apple-touch-icon-57x57.png"><link rel="apple-touch-icon" sizes="72x72" href="/img/favicons/apple-touch-icon-72x72.png"><link rel="apple-touch-icon" sizes="114x114" href="/img/favicons/apple-touch-icon-114x114.png"><link rel="manifest" href="/manifest.json"><link rel="prec